In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-43]) 
sys.path.append(cpath[0][:-43]+'/algorithms')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score, mean_absolute_error

from anchor import anchor_tabular

In [3]:
wine_q = Dataset()
X, y, feature_names, class_names = wine_q.load_wine_quality()

In [4]:
y.std()

0.8731880644450568

In [5]:
y = np.array(y)
#one_h_percent = int(min(10*len(X)/100,100))
#print("Instances:",one_h_percent)
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn
feature_names

['fixed_acidity',
 'volatile_acidity',
 'citric_acid',
 'residual_sugar',
 'chlorides',
 'free_sulfur_dioxide',
 'total_sulfur_dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [10]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names):
    parameters = [{
            'max_depth': [10],#10
            'max_features': [0.75],
            'bootstrap': [True],
            'min_samples_leaf' : [5],
            'n_estimators': [1000]#1000
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        """
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)
        #print('    GT and LT Ready')

        #DefragTechnique ================================================================================
        Kmax = 5
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        #print('    Done Splitting Starting Defrag')
        #mdl = DefragModel(modeltype=task, maxitr=10, qitr=0, tol=1e-6, restart=2, verbose=0, njobs=7)
        mdl = DefragModel(modeltype=task, maxitr=5, qitr=0, tol=1e-6, restart=2, njobs=6)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            return rule, cover, def_predictions, time.time() - ts
        #print('    Defrag Ready')
        """
        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance, instance_qe=0.4, method='R1')
            error = temp[-1]
            rule = {}
            for key,value in temp[5].items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule, error
        print('    LF Ready')

        #return {'gs':gt.rule,'ls':lt.rule,'lf':lf_rule, 'df': def_cov}
        return {'lf':lf_rule}
        
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'regression')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        #covered = False
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        #covered = False
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        #covered = False
                        return 0
        
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'lf':0, 'df':0}
    rule_length = {'gs':0, 'ls':0, 'lf':0, 'df':0}
    f_mae = {'gs':[], 'ls':[], 'lf':[], 'df':[]}
    time_response = {'gs':[], 'ls':[], 'lf':[], 'df':[]}
    
    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)    
    ktime = time.time()
    for test_ind in range(len(test)):
        if test_ind % 100 ==0:
            print(round(test_ind/(len(test))*100,2),'in:', time.time()-ktime)
        for name, method in rule_generator.items():
            if name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te = method(np.concatenate((x_test_temp[test_ind:test_ind+1],x_test_temp)))
                f_mae[name].append(mean_absolute_error(predictions,y_test_temp))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                ts = time.time()
                rule, error = method(x_test_temp_lf[test_ind])
                te = time.time() - ts
                coverage = 0
                mae = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        if str(error) != 'nan' and str(error) != 'None':
                            mae.append(error)
                    co = co + 1
                if len(mae) >= 1:
                    mae = np.array(mae)
                    f_mae[name].append(mae.mean())
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[test_ind])
                te = time.time() - ts
                coverage = 0
                error = []
                co = 0 
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    error.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(error) >= 1:
                    error = np.array(error)
                    f_mae[name].append(mean_absolute_error(error[:,:1],error[:,1:]))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_mae, time_response

In [11]:
def print_current(total_results):
    full_coverage = {'gs':[], 'ls':[], 'lf':[], 'df':[]}
    rule_length = {'gs':[], 'ls':[], 'lf':[], 'df':[]}
    f_mae = {'gs':[], 'ls':[], 'lf':[], 'df':[]}
    f_time = {'gs':[], 'ls':[], 'lf':[], 'df':[]}

    for i in total_results:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/len(i[4][name]))
            rule_length[name].append(i[2][name]/len(i[4][name]))
            if len(i[3][name]) >= 1:
                f_mae[name].append(np.array(i[3][name]))                
            f_time[name].append(np.array(i[4][name]))
    for name, method in i[0].items():
        print(name,np.array(full_coverage[name]).mean(),',',
              np.array(rule_length[name]).mean(),',',
              np.array(f_mae[name]).mean(),',',
              np.array(f_time[name]).mean())

In [12]:
total_results = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)
print(len(X_test))
results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
total_results.append(results)
total_results.append(results)
print_current(total_results)

1300
    LF Ready
0.0 in: 0.0
7.69 in: 121.96403098106384
15.38 in: 250.53716921806335
23.08 in: 373.9886209964752
30.77 in: 496.06824135780334
38.46 in: 630.405529499054
46.15 in: 759.5811223983765
53.85 in: 894.5463979244232
61.54 in: 1025.4812450408936
69.23 in: 1148.8560132980347
76.92 in: 1276.8090975284576
84.62 in: 1403.592509508133
92.31 in: 1540.5783298015594
lf 0.0008485207100591617 , 10.638461538461538 , 0.3986348667426222 , 1.2882218006940989


In [8]:
total_results = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)
print(len(X_test))
results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
total_results.append(results)
total_results.append(results)
print_current(total_results)

1300
    LF Ready
0.0 in: 0.0
7.69 in: 1599.366648197174
15.38 in: 3179.869612932205
23.08 in: 4786.340198516846
30.77 in: 6359.489060640335
38.46 in: 7936.766558885574
46.15 in: 9522.823098421097
53.85 in: 11113.280806303024
61.54 in: 12693.969573497772
69.23 in: 14268.842652320862
76.92 in: 15857.335349082947
84.62 in: 17444.499817848206
92.31 in: 19040.428656101227
gs 0.0011384615384615422 , 7.96076923076923 , 0.5985706910035996 , 0.00034943764026348407
ls 0.04787928994082837 , 4.49 , 0.5933948989479997 , 2.7971229610076316
lf 0.0008455621301775045 , 10.78 , 0.24855276016437294 , 1637623016.7856736
df 0.999230769230752 , 6.906153846153846 , 0.4178174165042527 , 11.894804888872
